# Processamento na GPU [![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://githubtocolab.com/chcomin/curso-visao-computacional-2024/blob/main/M06_classificacao_de_imagens_naturais/3%20-%20Processamento%20na%20GPU%20\(GPU\).ipynb)

### Copiando dados entre CPU e GPU

In [14]:
import torch
from torch import nn
from torchvision import models

x = torch.rand(16,3,224,224)
model = models.resnet18()
print(x.device)
print(model.conv1.weight.device)

cpu
cpu


In [15]:
# Move o tensor para a GPU
x_cuda = x.to('cuda')
# Move todos os tensores de parâmetros do modelo para a GPU. Note que como `modelo`
# é um objeto, os atributos dele é que são modificados. A referência para o modelo na
# CPU é perdida
model.to('cuda')

print(x.device)
print(x_cuda.device)
print(model.conv1.weight.device)

cpu
cuda:0
cuda:0


Com os dados e o modelo na GPU, é possível aplicar o modelo normalmente:

In [16]:
y = model(x_cuda)

Portanto, um loop de treinamento padrão é feito da seguinte forma:

In [17]:
device = 'cuda'  # Pode ser 'cuda' ou 'cpu'

# Dataloader artificial só para ilustração
target = torch.zeros(16, dtype=torch.long)
dl = [(x,target)]*10
optim = torch.optim.SGD(model.parameters(), lr=0.01)
loss_func = nn.CrossEntropyLoss()

loss_log = 0.
for imgs, targets in dl:
    imgs = imgs.to(device)
    targets = targets.to(device)
    model.zero_grad()
    scores = model(imgs)
    loss = loss_func(scores, targets)
    loss.backward()
    optim.step()

    # Como já vimos, é útil armazenar os valores da loss para plotar. Por isso
    # aplicamos um detach() para remoção do grafo de computação. Mas é importante
    # tomar cuidado porque que o valor ainda está na GPU
    loss_log += loss.detach()
# Podemos copiar o valor de volta para a CPU após processar todos os batches
loss_log = loss_log.to('cpu')

### Programação assíncrona

É importante notar que a execução na CPU e na GPU é feita de forma assíncrona, isto é, enquanto a GPU está processando os dados, o programa continua executando na CPU. Vamos entender isso na prática.

In [18]:
import time
import torch

# 100 matrizes de tamanho 900 x 900
x = torch.randn(100, 900, 900, device='cuda')

ti = time.perf_counter()
for i in range(500):
    # Multiplicação das 100 matrizes
    y = torch.matmul(x, x)
tt = time.perf_counter() - ti
print(tt)

0.017026599998644087


O código acima executa extremamente rápido! 

...será? Veja o que acontece se repetirmos exatamente o mesmo código mas mandarmos imprimir um valor do resultado:

In [19]:
import time
import torch

x = torch.randn(100, 900, 900, device='cuda')

ti = time.perf_counter()
for i in range(500):
    y = torch.matmul(x, x)
# Tempo de execução do loop
t_loop = time.perf_counter() - ti
print(y[0,0,0])
# Tempo de execução do loop + print
t_print = time.perf_counter() - ti
print(t_loop, t_print)

tensor(9.3082, device='cuda:0')
0.01712229999975534 10.816359999997076


O tempo medido após o print é muito maior do que o medido logo antes do print! Por acaso o print demorou para executar? Não, o que aconteceu é que a impressão de um valor de `y` é uma tarefa que bloqueia a CPU. O processo fica esperando a GPU terminar os cálculos para poder imprimir o valor.

O fato da execução na CPU e GPU serem assíncronas possui duas implicações importantes:

* Para garantir a máxima performance, é preciso tomar cuidado com operações que bloqueiam a CPU. Cópias entre a CPU e GPU bloqueiam, assim como impressão de valores na tela.
* É preciso tomar cuidado ao medir o tempo de execução de um código. O tempo é medido no processo da CPU, e não necessariamente leva em conta o tempo de execução na GPU

Copiar o resultado da GPU para a CPU bloqueia:

In [20]:
x = torch.randn(100, 900, 900, device='cuda')

ti = time.perf_counter()
for i in range(500):
    y = torch.matmul(x, x)
    loss = y.sum()
    loss = loss.to('cpu')
    # O mesmo ocorreria com os comandos
    #loss = loss.item()
    #print(loss)
tt = time.perf_counter() - ti
print(tt)

5.706739000001107


### Medindo performance

Vamos ver técnicas simples para medir o tempo de execução na CPU e GPU e o uso de memória na GPU

In [21]:
import torch
import time

def proc():
    '''Processamento a ser executado'''
    x = torch.randn(100, 900, 900, device='cuda')
    for i in range(500):
        y = torch.matmul(x, x)

def benchmark(func):

    # Eventos de medida de tempo na GPU
    gpu_start = torch.cuda.Event(enable_timing=True)
    gpu_end = torch.cuda.Event(enable_timing=True)  
    # Apaga registro de pico de memória
    torch.cuda.reset_peak_memory_stats()

    # Tempo inicial na CPU
    ti = time.perf_counter()
    # Envia um comando para a GPU para registrar o tempo
    gpu_start.record() 
    func()
    # Tempo final na CPU
    t_cpu = time.perf_counter() - ti
    # Tempo final na GPU
    gpu_end.record()
    # Bloqueia a CPU para esperar a GPU terminar os cálculos
    torch.cuda.synchronize()

    t_gpu = gpu_start.elapsed_time(gpu_end)/1000
    max_memory = torch.cuda.max_memory_allocated()/2**30

    return t_cpu, t_gpu, max_memory

t1, t2, m = benchmark(proc)
print(f'{t1:.3f}s, {t2:.3f}s, {m:.1f}GiB')

0.015s, 5.424s, 1.6GiB


### Mantendo a GPU ocupada (throughput)

Acima medimos o tempo de execução, mas também é importante verificarmos se a GPU está sendo usada de forma efetiva. Uma GPU possui milhares de cores, que estão organizados nos chamados *streaming multiprocessors (SM)*. Por exemplo, uma RTX 3080 12 GB possui 70 SMs, e cada SM possui:
1. 128 cores para operações de ponto flutuante de 32 bits (FP32);
2. 2 cores para operações de ponto flutuante de 64 bits;
3. 64 cores para operações com inteiro 32 bits;
4. 4 tensor cores

Dependendo da operação, cores distintos serão usados pelo SM. Em operações FP32, a RTX 3080 12 GB possui 128*70 = 8960 cores FP32. Portanto, a cada ciclo de clock podem ser realizadas 8960 operações em paralelo. Essa GPU trabalha em uma frequência máxima de 1710 MHz. Portanto, ela consegue realizar 15.32 TFLOPS operações por segundo. Esse é chamado de *throughput* máximo. A situação ideal é termos 100% de uso dos cores a todo momento. Uma forma de verificar o uso da GPU é através do comando `nvidia-smi dmon` no terminal. Mas esse comando pode dar resultados imprecisos. O nvidia-smi em geral mede a cada segundo a fração de tempo que **ao menos um SM esteve ativo**. Por exemplo, 100% de uso da GPU pode significar:

1. A GPU utilizou 100% dos SMs no período
2. A GPU utilizou apenas 1 SM no período

Para ilustrar a situação, executando o seguinte código em uma RTX 3080:

```python
    x = torch.tensor(0., device='cuda')
    for i in range(100000):
        x = x + 1.
```
o nvidia-smi mostrou 35% de uso da GPU. Mas esse código é completamente sequencial. A cada instante apenas um único core dentre os 8960 realiza uma operação. Então a GPU ficou praticamente sem uso.